<a href="https://colab.research.google.com/github/FXBY/ML-and-AI/blob/main/QRNN_Dev_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow-quantum==0.7.3

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
import cirq
import sympy
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from google.colab import files

upload = files.upload()
predata = pd.read_csv('data.csv')
predata = predata[['Year', 'Value']]
predata = predata.rename(columns={'Year':'Year', 'Value':'Value'})

data = predata.groupby('Year')['Value'].mean().reset_index()

train = data[data['Year'] < 2011]
test = data[data['Year'] >= 2011]

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

x_train = train[:, :-1]
y_train = train[:, -1]
x_test = test[:, :-1]
y_test = test[:, -1]

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

control_params = sympy.symbols('theta_1 theta_2 theta_3')

qubit = cirq.GridQubit(0, 0)
model_circuit = cirq.Circuit(
    cirq.rz(control_params[0])(qubit),
    cirq.ry(control_params[1])(qubit),
    cirq.rx(control_params[2])(qubit))

controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])
controller(tf.constant([[0.0],[1.0]])).numpy()

circuits_input = tf.keras.Input(shape=(),
                                dtype=tf.string,
                                name='circuits_input')

commands_input = tf.keras.Input(shape=(1,),
                                dtype=tf.dtypes.float32,
                                name='commands_input')

dense_2 = controller(commands_input)

expectation_layer = tfq.layers.ControlledPQC(model_circuit,
                                             operators = cirq.Z(qubit))
expectation = expectation_layer([circuits_input, dense_2])

model = tf.keras.Model(inputs=[circuits_input, commands_input],
                       outputs=expectation)

random_rotations = np.random.uniform(0, 2 * np.pi, 3)
noisy_preparation = cirq.Circuit(
  cirq.rx(random_rotations[0])(qubit),
  cirq.ry(random_rotations[1])(qubit),
  cirq.rz(random_rotations[2])(qubit)
)
datapoint_circuits = tfq.convert_to_tensor([
  noisy_preparation
] * 2)

model([datapoint_circuits, x_train]).numpy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit([datapoint_circuits, x_train],
                    y_train,
                    epochs=50,
                    verbose=0)

y_pred = model.predict(x_test)

plt.plot(data['Year'], data['Value'])
plt.plot(x_test[1], y_pred[1])
plt.title("Actual vs Predicted Values by the RNN Model")
plt.xlabel("Years")
plt.ylabel("CO2 Emissions (ppm)")
plt.grid(True)
plt.legend(['Real Data', 'Predicted Values'])
plt.xlim(data['Year'].min(), data['Year'].max())
plt.ylim(data['Value'].min(), data['Value'].max())
plt.show()

y_test = np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 1))))
y_pred = np.hstack((y_pred.reshape(-1, 1), np.zeros((y_pred.shape[0], 1))))

y_test = scaler.inverse_transform(y_test)
y_pred = scaler.inverse_transform(y_pred)

rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print('Test RMSE: %.2f' % rmse)